## Getting the Data

In [131]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
data = pd.read_csv(r'../data/final_competition_data.csv')

In [108]:
data.head()

,date,url,caption
0,2025-11-25,https://instagram.com/p/DRdk404kyhv,📢 Giveaway Alert!\n\n🎉 CS-Related Competitions...
1,2025-12-09,https://instagram.com/p/DSBr6pIk5w7,The annual event from ISC IDN Akhwat yang kali...
2,2025-12-04,https://instagram.com/p/DR0u-QjE1U2,🔐 INFENTRA CTF COMPETITION 2025 🔐\n\nTunjukkan...
3,2025-12-14,https://instagram.com/p/DSOzq4rk3bF,[GARENA GAME JAM 3: REINVENT THE GAME]\n\nGare...
4,2025-12-14,https://instagram.com/p/3787191574871667741,NaN


In [109]:
data.shape

(2523, 3)

In [110]:
data.date.describe()

count           2523
unique           593
top       2025-07-26
freq              35
Name: date, dtype: object

In [111]:
print(f'Tanggal post terakhir {data.date.max()}')
print(f'Tanggal post terawal {data.date.min()}')

Tanggal post terakhir 2025-12-14
Tanggal post terawal 2022-09-06


In [112]:
# dropping the url column since were not going to use that
data = data.drop(['url'], axis=1).reset_index(drop=True)
data.head()

,date,caption
0,2025-11-25,📢 Giveaway Alert!\n\n🎉 CS-Related Competitions...
1,2025-12-09,The annual event from ISC IDN Akhwat yang kali...
2,2025-12-04,🔐 INFENTRA CTF COMPETITION 2025 🔐\n\nTunjukkan...
3,2025-12-14,[GARENA GAME JAM 3: REINVENT THE GAME]\n\nGare...
4,2025-12-14,NaN


Keeping only the posts from the year 2023 to 2025

In [113]:
# extracting the day, month, and year from the columns date
data['date'] = pd.to_datetime(data['date'], errors='coerce')

data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

data.head()

,date,caption,day,month,year
0,2025-11-25,📢 Giveaway Alert!\n\n🎉 CS-Related Competitions...,25,11,2025
1,2025-12-09,The annual event from ISC IDN Akhwat yang kali...,9,12,2025
2,2025-12-04,🔐 INFENTRA CTF COMPETITION 2025 🔐\n\nTunjukkan...,4,12,2025
3,2025-12-14,[GARENA GAME JAM 3: REINVENT THE GAME]\n\nGare...,14,12,2025
4,2025-12-14,NaN,14,12,2025


In [114]:
# cutting the year that are not from 2023-2025
data = data.loc[(data['year'] >= 2023) & (data['year'] <= 2025)]

In [115]:
print(f'Data shape after cutting the data that are not from 2023-2025: {data.shape}')

Data shape after cutting the data that are not from 2023-2025: (2434, 5)


Dropping columns:
- that have null value in the column caption since that entries gonna be useless
- that duplicate

In [116]:
data.isna().sum()

date          0
caption    1121
day           0
month         0
year          0
dtype: int64

In [117]:
# dropping entries that have null caption 
n = data.loc[data['caption'].isna()].shape[0]
print(f'Number of entries that have null caption: {n}')

print(f'Total number of entries before: {data.shape[0]}')

data = data.loc[~data['caption'].isna()].reset_index(drop=True)
print(f'Total number of entries after: {data.shape[0]}')

Number of entries that have null caption: 1121
Total number of entries before: 2434
Total number of entries after: 1313


In [118]:
# dropping entries that are duplicated
data.duplicated().shape

(1313,)

No duplicated entries

In [119]:
# final look at the data before going into the heavy preprocessing
data.head(20)

,date,caption,day,month,year
0,2025-11-25,📢 Giveaway Alert!\n\n🎉 CS-Related Competitions...,25,11,2025
1,2025-12-09,The annual event from ISC IDN Akhwat yang kali...,9,12,2025
2,2025-12-04,🔐 INFENTRA CTF COMPETITION 2025 🔐\n\nTunjukkan...,4,12,2025
3,2025-12-14,[GARENA GAME JAM 3: REINVENT THE GAME]\n\nGare...,14,12,2025
4,2025-12-14,🌍 Prog{r}amming 8.2 is BACK! 🌿\n\nProg{r}ammin...,14,12,2025
5,2025-12-14,Panggilan buat semua pelajar! 📢 Saatnya tunjuk...,14,12,2025
6,2025-12-11,HEY HEY🙌🏻🚨ATTENTION🚨\nGet READY! because Tensa...,11,12,2025
7,2025-12-11,BASE INDONESIA HACKATHON IS LIVE!\nGot a kille...,11,12,2025
8,2025-12-09,🛰️ Peluncuran Resmi: ASCEND – AEROO Satellite ...,9,12,2025
9,2025-12-09,Bangun Solusi AI yang Bisa Langsung Kamu Pajan...,9,12,2025


In [122]:
print('Post #1')
print(data.caption[0])

print('\n===============================\n')

print('Post #2')
print(data.caption[100])

print('\n===============================\n')

print('Post #3')
print(data.caption[1000])

print('\n===============================\n')

print('Post #4')
print(data.caption[1300])

Post #1
📢 Giveaway Alert!

🎉 CS-Related Competitions has reached 13.000 followers! 🎉 

To celebrate this milestone, we are giving away 3 prizes!

🎁 Laptop Backpack
Limited-edition, from Olimpiade Sains Nasional (OSN) 2025 🇮🇩
*Donated by A****** M*****, thanks bro

🎁 Insulated Mug with Lid 
Limited-edition, from the 2025 International Collegiate Programming Contest (ICPC) World Finals held in Azerbaijan 🇦🇿

🎁 Metal Keychain
Limited-edition, from the European Girls Olympiad in Informatics (EGOI) 2024 held in the Netherlands 🇳🇱

Details:
- Shipping is included in the giveaway.
- All winners will also get a plain notebook from @csrelatedcompetitions.
- All merchandise is brand new and official from the competition (not sponsored)

✨ How to enter (for Indonesian residents only), get 1 entry:
1. Follow @csrelatedcompetitions
2. Like this post ♥️
3. Tag two (or more) friends in the comments
4. Answer (in English or bahasa Indonesia): 

❝❝❝
What Computer Science competition field do you usuall

## Start Prepocessing and Feature Engineering

### Getting clean text of the caption

In [124]:
import re

df = data.copy()

# text cleaning
def clean_caption(text):
    if not isinstance(text, str): return ""
    
    # make it to be one single line
    text = re.sub(r'\n+', ' ', text)
    
    # remove "Link in bio", as they are usually noise
    text = re.sub(r'(?i)link\s+in\s+bio', '', text)

    # remove emoji from the text
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # strip leading/trailing whitespace
    return text.strip()

# getting the hashtags and mentions
def get_hashtags(text):
    if not isinstance(text, str): return []
    return re.findall(r'#(\w+)', text)

def get_mentions(text):
    if not isinstance(text, str): return []
    return re.findall(r'@(\w+)', text)


df['clean_text'] = df['caption'].apply(clean_caption)
df['hashtags'] = df['caption'].apply(get_hashtags)
df['mentions'] = df['caption'].apply(get_mentions)

data = df.copy()

df[['date', 'hashtags', 'clean_text']].head()

,date,hashtags,clean_text
0,2025-11-25,[],Giveaway Alert! CS-Related Competitions has r...
1,2025-12-09,[],The annual event from ISC IDN Akhwat yang kali...
2,2025-12-04,[],INFENTRA CTF COMPETITION 2025 Tunjukkan keahl...
3,2025-12-14,"[gamejam, hackathonindonesia, garena, lombait,...",[GARENA GAME JAM 3: REINVENT THE GAME] Garena ...
4,2025-12-14,[],Prog{r}amming 8.2 is BACK! Prog{r}amming 8.2 ...


### Extracting price

In [129]:
df = data.copy()

def extract_price_tiers(text):
    if not isinstance(text, str):
        return pd.Series([None, None, None])
    
    text_lower = text.lower()
    
    # 1. FIND ALL POTENTIAL PRICES
    found_prices = []
    
    # Check for "Gratis" / "Free"
    if 'gratis' in text_lower or 'free registration' in text_lower:
        found_prices.append(0)
    
    # Pattern A: "Rp 50.000" or "Rp50.000" or "50.000"
    # Matches 'rp' followed by numbers with dots/commas
    matches_rp = re.findall(r'(?:rp\.?|idr)\s?([\d\.,]+)', text_lower)
    for m in matches_rp:
        clean_num = m.replace('.', '').replace(',', '')
        if clean_num.isdigit():
            val = int(clean_num)
            # Filter out noise: Prices usually aren't less than 1000 rupiah (unless 0)
            # and usually not part of a phone number (checking length helps, but heuristic)
            if val > 1000 or val == 0: 
                found_prices.append(val)

    # Pattern B: "50k", "100k", "150k"
    matches_k = re.findall(r'\b(\d+)k\b', text_lower)
    for m in matches_k:
        if m.isdigit():
            found_prices.append(int(m) * 1000)
            
    # 2. CLEAN UP & SORT
    # Remove duplicates and sort Ascending (Smallest -> Largest)
    unique_prices = sorted(list(set(found_prices)))
    
    # 3. ASSIGN TO BATCHES
    # Logic: 
    # Batch 1 = Lowest Price
    # Batch 2 = Second Lowest
    # Batch 3 = Third Lowest (if any)
    
    p1 = unique_prices[0] if len(unique_prices) > 0 else None
    p2 = unique_prices[1] if len(unique_prices) > 1 else None
    p3 = unique_prices[2] if len(unique_prices) > 2 else None
    
    return pd.Series([p1, p2, p3])


df[['price_batch_1', 'price_batch_2', 'price_batch_3']] = df['caption'].apply(extract_price_tiers)
display(df[['date', 'clean_text', 'price_batch_1', 'price_batch_2', 'price_batch_3']].head(10))

data = df.copy()

,date,clean_text,price_batch_1,price_batch_2,price_batch_3
0,2025-11-25,Giveaway Alert! CS-Related Competitions has r...,NaN,NaN,NaN
1,2025-12-09,The annual event from ISC IDN Akhwat yang kali...,NaN,NaN,NaN
2,2025-12-04,INFENTRA CTF COMPETITION 2025 Tunjukkan keahl...,50000.0,75000.0,NaN
3,2025-12-14,[GARENA GAME JAM 3: REINVENT THE GAME] Garena ...,NaN,NaN,NaN
4,2025-12-14,Prog{r}amming 8.2 is BACK! Prog{r}amming 8.2 ...,NaN,NaN,NaN
5,2025-12-14,Panggilan buat semua pelajar! Saatnya tunjukk...,NaN,NaN,NaN
6,2025-12-11,HEY HEYATTENTION Get READY! because Tensailici...,60000.0,65000.0,70000.0
7,2025-12-11,BASE INDONESIA HACKATHON IS LIVE! Got a killer...,NaN,NaN,NaN
8,2025-12-09,Peluncuran Resmi: ASCEND AEROO Satellite Chal...,NaN,NaN,NaN
9,2025-12-09,Bangun Solusi AI yang Bisa Langsung Kamu Pajan...,0.0,NaN,NaN


In [130]:
data.head()

,date,caption,day,month,year,clean_text,hashtags,mentions,price_batch_1,price_batch_2,price_batch_3
0,2025-11-25,📢 Giveaway Alert!\n\n🎉 CS-Related Competitions...,25,11,2025,Giveaway Alert! CS-Related Competitions has r...,[],"[csrelatedcompetitions, csrelatedcompetitions,...",NaN,NaN,NaN
1,2025-12-09,The annual event from ISC IDN Akhwat yang kali...,9,12,2025,The annual event from ISC IDN Akhwat yang kali...,[],[],NaN,NaN,NaN
2,2025-12-04,🔐 INFENTRA CTF COMPETITION 2025 🔐\n\nTunjukkan...,4,12,2025,INFENTRA CTF COMPETITION 2025 Tunjukkan keahl...,[],[],50000.0,75000.0,NaN
3,2025-12-14,[GARENA GAME JAM 3: REINVENT THE GAME]\n\nGare...,14,12,2025,[GARENA GAME JAM 3: REINVENT THE GAME] Garena ...,"[gamejam, hackathonindonesia, garena, lombait,...",[],NaN,NaN,NaN
4,2025-12-14,🌍 Prog{r}amming 8.2 is BACK! 🌿\n\nProg{r}ammin...,14,12,2025,Prog{r}amming 8.2 is BACK! Prog{r}amming 8.2 ...,[],[programming8],NaN,NaN,NaN


## Saving Currently Data to CSV

In [165]:
SAVING_PATH = r'..\data\final_competition_data_1cleaned.csv'
data.to_csv(SAVING_PATH)

In [166]:
import pandas as pd

data_cleaned = pd.read_csv(r'..\data\final_competition_data_1cleaned.csv')

In [167]:
data_cleaned.head()

,Unnamed: 0,date,caption,day,month,year,clean_text,hashtags,mentions,price_batch_1,price_batch_2,price_batch_3
0,0,2025-11-25,📢 Giveaway Alert!\n\n🎉 CS-Related Competitions...,25,11,2025,Giveaway Alert! CS-Related Competitions has r...,[],"['csrelatedcompetitions', 'csrelatedcompetitio...",NaN,NaN,NaN
1,1,2025-12-09,The annual event from ISC IDN Akhwat yang kali...,9,12,2025,The annual event from ISC IDN Akhwat yang kali...,[],[],NaN,NaN,NaN
2,2,2025-12-04,🔐 INFENTRA CTF COMPETITION 2025 🔐\n\nTunjukkan...,4,12,2025,INFENTRA CTF COMPETITION 2025 Tunjukkan keahl...,[],[],50000.0,75000.0,NaN
3,3,2025-12-14,[GARENA GAME JAM 3: REINVENT THE GAME]\n\nGare...,14,12,2025,[GARENA GAME JAM 3: REINVENT THE GAME] Garena ...,"['gamejam', 'hackathonindonesia', 'garena', 'l...",[],NaN,NaN,NaN
4,4,2025-12-14,🌍 Prog{r}amming 8.2 is BACK! 🌿\n\nProg{r}ammin...,14,12,2025,Prog{r}amming 8.2 is BACK! Prog{r}amming 8.2 ...,[],['programming8'],NaN,NaN,NaN


In [193]:
print(data_cleaned.shape)
print(data_cleaned.loc[data_cleaned.caption.apply(lambda x: len(x)>0)].shape)
print(data_cleaned.loc[data_cleaned.caption.apply(lambda x: len(x)<10)].shape)
print(data_cleaned.loc[data_cleaned.caption.apply(lambda x: len(x)<20)].shape)

(1313, 12)
(1313, 12)
(1, 12)
(19, 12)


In [192]:
data_cleaned.loc[data_cleaned.caption.apply(lambda x: len(x)<20)]

,Unnamed: 0,date,caption,day,month,year,clean_text,hashtags,mentions,price_batch_1,price_batch_2,price_batch_3
969,969,2024-01-16,https://mitit.org/,16,1,2024,https://mitit.org/,[],[],NaN,NaN,NaN
982,982,2024-01-07,@himasiunipdu,7,1,2024,@himasiunipdu,[],['himasiunipdu'],NaN,NaN,NaN
1011,1011,2023-11-29,https://tuctf.com/,29,11,2023,https://tuctf.com/,[],[],NaN,NaN,NaN
1140,1140,2023-08-04,@ifestunpad,4,8,2023,@ifestunpad,[],['ifestunpad'],NaN,NaN,NaN
1176,1176,2023-07-11,@eltrumstechnorace,11,7,2023,@eltrumstechnorace,[],['eltrumstechnorace'],NaN,NaN,NaN
1182,1182,2023-07-10,@itconvert_unej,10,7,2023,@itconvert_unej,[],['itconvert_unej'],NaN,NaN,NaN
1186,1186,2023-07-07,@edufair.himsika,7,7,2023,@edufair.himsika,[],['edufair'],NaN,NaN,NaN
1208,1208,2023-06-18,@amcc.code,18,6,2023,@amcc.code,[],['amcc'],NaN,NaN,NaN
1217,1217,2023-06-02,@techfest.himti,2,6,2023,@techfest.himti,[],['techfest'],NaN,NaN,NaN
1224,1224,2023-05-26,@ayoconnect.id,26,5,2023,@ayoconnect.id,[],['ayoconnect'],NaN,NaN,NaN
